5 LSTM layer, bi-directional, 4 linear layer, 512 hidden size, add dropout

In [2]:
PHONEME_LIST = [
    "+BREATH+",
    "+COUGH+",
    "+NOISE+",
    "+SMACK+",
    "+UH+",
    "+UM+",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "HH",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "SIL",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

num_classes = len(PHONEME_LIST) + 1
print(num_classes) 

47


In [3]:
PHONEME_MAP = [
    '_',  # "+BREATH+"
    '+',  # "+COUGH+"
    '~',  # "+NOISE+"
    '!',  # "+SMACK+"
    '-',  # "+UH+"
    '@',  # "+UM+"
    'a',  # "AA"
    'A',  # "AE"
    'h',  # "AH"
    'o',  # "AO"
    'w',  # "AW"
    'y',  # "AY"
    'b',  # "B"
    'c',  # "CH"
    'd',  # "D"
    'D',  # "DH"
    'e',  # "EH"
    'r',  # "ER"
    'E',  # "EY"
    'f',  # "F"
    'g',  # "G"
    'H',  # "HH"
    'i',  # "IH"
    'I',  # "IY"
    'j',  # "JH"
    'k',  # "K"
    'l',  # "L"
    'm',  # "M"
    'n',  # "N"
    'G',  # "NG"
    'O',  # "OW"
    'Y',  # "OY"
    'p',  # "P"
    'R',  # "R"
    's',  # "S"
    'S',  # "SH"
    '.',  # "SIL"
    't',  # "T"
    'T',  # "TH"
    'u',  # "UH"
    'U',  # "UW"
    'v',  # "V"
    'W',  # "W"
    '?',  # "Y"
    'z',  # "Z"
    'Z',  # "ZH"
]

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import tarfile
import zipfile

with zipfile.ZipFile('/content/gdrive/My Drive/11785hw3/homework-3-part-2-11-785-fall-2019.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [5]:
!ls HW3P2_Data

phoneme_list.py		    wsj0_dev.npy		  wsj0_train.npy
sample_submission.csv	    wsj0_test.npy
wsj0_dev_merged_labels.npy  wsj0_train_merged_labels.npy


In [4]:
import numpy as np
import sys
import os
import torch
import torch.nn as nn
from torch.nn.utils.rnn import *
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import time

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cuda

True

In [5]:
class WSJ():
    """ Load the WSJ speech dataset
        
        Ensure WSJ_PATH is path to directory containing 
        all data files (.npy) provided on Kaggle.
            
    """
  
    def __init__(self):
        self.dev_set = None
        self.train_set = None
        self.test_set = None
  
    @property
    def dev(self):
        if self.dev_set is None:
            self.dev_set = load_raw(os.environ['WSJ_PATH'], 'wsj0_dev')
        return self.dev_set

    @property
    def train(self):
        if self.train_set is None:
            self.train_set = load_raw(os.environ['WSJ_PATH'], 'wsj0_train')
        return self.train_set
  
    @property
    def test(self):
        if self.test_set is None:
            self.test_set = (np.load(os.path.join(os.environ['WSJ_PATH'], 'wsj0_test.npy'), encoding='bytes',allow_pickle=True), None)
        return self.test_set
    
def load_raw(path, name):
    return (
        np.load(os.path.join(path, '{}.npy'.format(name)), encoding='bytes',allow_pickle=True), 
        np.load(os.path.join(path, '{}_merged_labels.npy'.format(name)), encoding='bytes',allow_pickle=True)
    )

os.environ['WSJ_PATH'] = 'HW3P2_Data'
loader = WSJ()
trainX, trainY = loader.train
print(trainX.shape)
evalX, evalY = loader.dev
print(evalX.shape)
testX, testY = loader.test
print(testX.shape)

(24724,)
(1106,)
(523,)


In [6]:
class Dataset(Dataset):
    def __init__(self, frames, labels):
        self.frames=[torch.tensor(f) for f in frames]
        self.labels = [torch.tensor(l) for l in labels]
    def __getitem__(self,i):
        time_steps = self.frames[i]
        labels_original = self.labels[i]
        labels_add1 = torch.LongTensor([label + 1 for label in labels_original])
        
        return time_steps.to(device), labels_add1.to(device)
    def __len__(self):
        return len(self.frames)

def collate_lines(seq_list):
    inputs, targets = zip(*seq_list)
    X_lens = torch.LongTensor([len(seq) for seq in inputs])
    Y_lens = torch.LongTensor([len(seq) for seq in targets])    
    X = pad_sequence(inputs)
    # batch_first = True is required for use in nn.CTCLoss.
    Y = pad_sequence(targets, batch_first=True)    
    return X, Y, X_lens, Y_lens

In [7]:
train_dataset = Dataset(trainX, trainY)
eval_dataset = Dataset(evalX, evalY)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, collate_fn = collate_lines)
eval_loader = DataLoader(eval_dataset, shuffle=True, batch_size=32, collate_fn = collate_lines)

In [8]:
class Network(nn.Module):
    
    def __init__(self, input_size, num_classes, hidden_size):
        super(Network, self).__init__()
        
        self.input_size = input_size
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size = input_size, hidden_size = hidden_size, dropout=0.5, num_layers=5, bidirectional=True) # defaule 1 layer, batch_first = False
        self.scoring1 = nn.Linear(hidden_size * 2, 2048)
        self.scoring2 = nn.Linear(2048, 1024)
        self.scoring3 = nn.Linear(1024, 512)
        self.scoring4 = nn.Linear(512, num_classes)
    
    def forward(self, seq_list, X_lens):

        packed_input = rnn.pack_padded_sequence(seq_list, X_lens, enforce_sorted=False)
        
        hidden = None
        output_packed, output_hidden = self.rnn(packed_input, hidden)
        output_padded, out_lens = rnn.pad_packed_sequence(output_packed)
        scores = self.scoring1(output_padded)
        scores = self.scoring2(scores)
        scores = self.scoring3(scores)
        scores = self.scoring4(scores)
        
        # Log softmax after output layer is required for use in `nn.CTCLoss`.
        out = scores.log_softmax(2)
        
        return out, out_lens

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'bias' in name:
                torch.nn.init.constant_(param, 0.0)
            elif 'weight' in name:
                torch.nn.init.xavier_normal_(param)   

In [9]:
def train_epoch_packed(model, train_loader, eval_loader):
  
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (inputs, targets, X_lens, Y_lens) in enumerate(train_loader):
            #inputs, targets, Y_lens = inputs.to(device), targets.to(device), Y_lens.to(device)
            optimizer.zero_grad()
            outputs, out_lens = model(inputs, X_lens)
            loss = criterion(outputs, targets, out_lens, Y_lens)
            loss.backward()
            optimizer.step()
      
            avg_loss += loss.item()
            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                print('Epoch: {}\tBatch: {}\tTraining perplexity: {:.4f}'.format(epoch+1, batch_num+1, np.exp(avg_loss/50)))
                avg_loss = 0.0
                
            torch.cuda.empty_cache()
            del inputs
            del targets
            del Y_lens
            del loss

        model.eval()
        val_loss = 0

        for batch_num, (inputs, targets, X_lens, Y_lens) in enumerate(eval_loader):

            outputs, out_lens = model(inputs, X_lens)
            loss = criterion(outputs, targets, out_lens, Y_lens)
            val_loss += loss.item()
            
        val_loss /= len(eval_loader)
        print("\nValidation loss:",val_loss)
        print("Validation perplexity :",np.exp(val_loss),"\n")
        model.train()

In [10]:
torch.manual_seed(11785)

learningRate = 1e-3

model = Network(40, num_classes, 512)
model.apply(init_weights)
#model = torch.load('hw3p2_5_dropout_epoch5.pth')

criterion = nn.CTCLoss()

criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

In [10]:
#epoch 5
numEpochs = 5
model.to(device)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 5.4373
Epoch: 1	Batch: 50	Training perplexity: 229.8291
Epoch: 1	Batch: 100	Avg-Loss: 3.4230
Epoch: 1	Batch: 100	Training perplexity: 30.6627
Epoch: 1	Batch: 150	Avg-Loss: 3.1679
Epoch: 1	Batch: 150	Training perplexity: 23.7566
Epoch: 1	Batch: 200	Avg-Loss: 2.9701
Epoch: 1	Batch: 200	Training perplexity: 19.4947
Epoch: 1	Batch: 250	Avg-Loss: 2.5878
Epoch: 1	Batch: 250	Training perplexity: 13.3002
Epoch: 1	Batch: 300	Avg-Loss: 2.1565
Epoch: 1	Batch: 300	Training perplexity: 8.6408
Epoch: 1	Batch: 350	Avg-Loss: 1.8159
Epoch: 1	Batch: 350	Training perplexity: 6.1463
Epoch: 1	Batch: 400	Avg-Loss: 1.5728
Epoch: 1	Batch: 400	Training perplexity: 4.8203
Epoch: 1	Batch: 450	Avg-Loss: 1.4003
Epoch: 1	Batch: 450	Training perplexity: 4.0565
Epoch: 1	Batch: 500	Avg-Loss: 1.3027
Epoch: 1	Batch: 500	Training perplexity: 3.6790
Epoch: 1	Batch: 550	Avg-Loss: 1.3082
Epoch: 1	Batch: 550	Training perplexity: 3.6995
Epoch: 1	Batch: 600	Avg-Loss: 1.1638
Epoch: 1	Batch: 600	Trai

In [10]:
#epoch 6
numEpochs = 1
learningRate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.5350
Epoch: 1	Batch: 50	Training perplexity: 1.7075
Epoch: 1	Batch: 100	Avg-Loss: 0.5199
Epoch: 1	Batch: 100	Training perplexity: 1.6819
Epoch: 1	Batch: 150	Avg-Loss: 0.5703
Epoch: 1	Batch: 150	Training perplexity: 1.7687
Epoch: 1	Batch: 200	Avg-Loss: 0.5603
Epoch: 1	Batch: 200	Training perplexity: 1.7512
Epoch: 1	Batch: 250	Avg-Loss: 0.5155
Epoch: 1	Batch: 250	Training perplexity: 1.6744
Epoch: 1	Batch: 300	Avg-Loss: 0.5216
Epoch: 1	Batch: 300	Training perplexity: 1.6847
Epoch: 1	Batch: 350	Avg-Loss: 0.4841
Epoch: 1	Batch: 350	Training perplexity: 1.6227
Epoch: 1	Batch: 400	Avg-Loss: 0.5135
Epoch: 1	Batch: 400	Training perplexity: 1.6712
Epoch: 1	Batch: 450	Avg-Loss: 0.4616
Epoch: 1	Batch: 450	Training perplexity: 1.5866
Epoch: 1	Batch: 500	Avg-Loss: 0.4759
Epoch: 1	Batch: 500	Training perplexity: 1.6095
Epoch: 1	Batch: 550	Avg-Loss: 0.4603
Epoch: 1	Batch: 550	Training perplexity: 1.5845
Epoch: 1	Batch: 600	Avg-Loss: 0.5175
Epoch: 1	Batch: 600	Training p

In [11]:
#epoch 7
numEpochs = 1
learningRate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.4628
Epoch: 1	Batch: 50	Training perplexity: 1.5886
Epoch: 1	Batch: 100	Avg-Loss: 0.4700
Epoch: 1	Batch: 100	Training perplexity: 1.5999
Epoch: 1	Batch: 150	Avg-Loss: 0.4785
Epoch: 1	Batch: 150	Training perplexity: 1.6137
Epoch: 1	Batch: 200	Avg-Loss: 0.4610
Epoch: 1	Batch: 200	Training perplexity: 1.5857
Epoch: 1	Batch: 250	Avg-Loss: 0.4618
Epoch: 1	Batch: 250	Training perplexity: 1.5870
Epoch: 1	Batch: 300	Avg-Loss: 0.4272
Epoch: 1	Batch: 300	Training perplexity: 1.5330
Epoch: 1	Batch: 350	Avg-Loss: 0.5190
Epoch: 1	Batch: 350	Training perplexity: 1.6803
Epoch: 1	Batch: 400	Avg-Loss: 0.5509
Epoch: 1	Batch: 400	Training perplexity: 1.7348
Epoch: 1	Batch: 450	Avg-Loss: 0.4645
Epoch: 1	Batch: 450	Training perplexity: 1.5911
Epoch: 1	Batch: 500	Avg-Loss: 0.4679
Epoch: 1	Batch: 500	Training perplexity: 1.5966
Epoch: 1	Batch: 550	Avg-Loss: 0.4653
Epoch: 1	Batch: 550	Training perplexity: 1.5926
Epoch: 1	Batch: 600	Avg-Loss: 0.4309
Epoch: 1	Batch: 600	Training p

In [12]:
#epoch 8
numEpochs = 1
learningRate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.3902
Epoch: 1	Batch: 50	Training perplexity: 1.4773
Epoch: 1	Batch: 100	Avg-Loss: 0.3708
Epoch: 1	Batch: 100	Training perplexity: 1.4489
Epoch: 1	Batch: 150	Avg-Loss: 0.3393
Epoch: 1	Batch: 150	Training perplexity: 1.4039
Epoch: 1	Batch: 200	Avg-Loss: 0.3612
Epoch: 1	Batch: 200	Training perplexity: 1.4350
Epoch: 1	Batch: 250	Avg-Loss: 0.3377
Epoch: 1	Batch: 250	Training perplexity: 1.4018
Epoch: 1	Batch: 300	Avg-Loss: 0.3359
Epoch: 1	Batch: 300	Training perplexity: 1.3992
Epoch: 1	Batch: 350	Avg-Loss: 0.3131
Epoch: 1	Batch: 350	Training perplexity: 1.3677
Epoch: 1	Batch: 400	Avg-Loss: 0.3676
Epoch: 1	Batch: 400	Training perplexity: 1.4443
Epoch: 1	Batch: 450	Avg-Loss: 0.3140
Epoch: 1	Batch: 450	Training perplexity: 1.3688
Epoch: 1	Batch: 500	Avg-Loss: 0.3290
Epoch: 1	Batch: 500	Training perplexity: 1.3896
Epoch: 1	Batch: 550	Avg-Loss: 0.3138
Epoch: 1	Batch: 550	Training perplexity: 1.3686
Epoch: 1	Batch: 600	Avg-Loss: 0.3443
Epoch: 1	Batch: 600	Training p

In [19]:
#epoch 9
numEpochs = 1
learningRate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.2871
Epoch: 1	Batch: 50	Training perplexity: 1.3326
Epoch: 1	Batch: 100	Avg-Loss: 0.3384
Epoch: 1	Batch: 100	Training perplexity: 1.4028
Epoch: 1	Batch: 150	Avg-Loss: 0.2914
Epoch: 1	Batch: 150	Training perplexity: 1.3383
Epoch: 1	Batch: 200	Avg-Loss: 0.3024
Epoch: 1	Batch: 200	Training perplexity: 1.3531
Epoch: 1	Batch: 250	Avg-Loss: 0.2708
Epoch: 1	Batch: 250	Training perplexity: 1.3111
Epoch: 1	Batch: 300	Avg-Loss: 0.3218
Epoch: 1	Batch: 300	Training perplexity: 1.3796
Epoch: 1	Batch: 350	Avg-Loss: 0.2865
Epoch: 1	Batch: 350	Training perplexity: 1.3317
Epoch: 1	Batch: 400	Avg-Loss: 0.2916
Epoch: 1	Batch: 400	Training perplexity: 1.3385
Epoch: 1	Batch: 450	Avg-Loss: 0.2840
Epoch: 1	Batch: 450	Training perplexity: 1.3285
Epoch: 1	Batch: 500	Avg-Loss: 0.2869
Epoch: 1	Batch: 500	Training perplexity: 1.3323
Epoch: 1	Batch: 550	Avg-Loss: 0.3229
Epoch: 1	Batch: 550	Training perplexity: 1.3811
Epoch: 1	Batch: 600	Avg-Loss: 0.3082
Epoch: 1	Batch: 600	Training p

In [22]:
#epoch 10
numEpochs = 1
learningRate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.2752
Epoch: 1	Batch: 50	Training perplexity: 1.3168
Epoch: 1	Batch: 100	Avg-Loss: 0.2901
Epoch: 1	Batch: 100	Training perplexity: 1.3366
Epoch: 1	Batch: 150	Avg-Loss: 0.2537
Epoch: 1	Batch: 150	Training perplexity: 1.2888
Epoch: 1	Batch: 200	Avg-Loss: 0.2716
Epoch: 1	Batch: 200	Training perplexity: 1.3120
Epoch: 1	Batch: 250	Avg-Loss: 0.2835
Epoch: 1	Batch: 250	Training perplexity: 1.3278
Epoch: 1	Batch: 300	Avg-Loss: 0.2897
Epoch: 1	Batch: 300	Training perplexity: 1.3360
Epoch: 1	Batch: 350	Avg-Loss: 0.2775
Epoch: 1	Batch: 350	Training perplexity: 1.3199
Epoch: 1	Batch: 400	Avg-Loss: 0.2624
Epoch: 1	Batch: 400	Training perplexity: 1.3001
Epoch: 1	Batch: 450	Avg-Loss: 0.2717
Epoch: 1	Batch: 450	Training perplexity: 1.3122
Epoch: 1	Batch: 500	Avg-Loss: 0.2780
Epoch: 1	Batch: 500	Training perplexity: 1.3205
Epoch: 1	Batch: 550	Avg-Loss: 0.2780
Epoch: 1	Batch: 550	Training perplexity: 1.3205
Epoch: 1	Batch: 600	Avg-Loss: 0.2845
Epoch: 1	Batch: 600	Training p

In [26]:
#epoch 11
numEpochs = 1
learningRate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.2572
Epoch: 1	Batch: 50	Training perplexity: 1.2933
Epoch: 1	Batch: 100	Avg-Loss: 0.2340
Epoch: 1	Batch: 100	Training perplexity: 1.2637
Epoch: 1	Batch: 150	Avg-Loss: 0.2568
Epoch: 1	Batch: 150	Training perplexity: 1.2928
Epoch: 1	Batch: 200	Avg-Loss: 0.2589
Epoch: 1	Batch: 200	Training perplexity: 1.2955
Epoch: 1	Batch: 250	Avg-Loss: 0.2744
Epoch: 1	Batch: 250	Training perplexity: 1.3158
Epoch: 1	Batch: 300	Avg-Loss: 0.2321
Epoch: 1	Batch: 300	Training perplexity: 1.2612
Epoch: 1	Batch: 350	Avg-Loss: 0.2742
Epoch: 1	Batch: 350	Training perplexity: 1.3154
Epoch: 1	Batch: 400	Avg-Loss: 0.2544
Epoch: 1	Batch: 400	Training perplexity: 1.2897
Epoch: 1	Batch: 450	Avg-Loss: 0.2579
Epoch: 1	Batch: 450	Training perplexity: 1.2942
Epoch: 1	Batch: 500	Avg-Loss: 0.2410
Epoch: 1	Batch: 500	Training perplexity: 1.2725
Epoch: 1	Batch: 550	Avg-Loss: 0.2555
Epoch: 1	Batch: 550	Training perplexity: 1.2911
Epoch: 1	Batch: 600	Avg-Loss: 0.2984
Epoch: 1	Batch: 600	Training p

In [11]:
#epoch 12
numEpochs = 1
learningRate = 3e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.2203
Epoch: 1	Batch: 50	Training perplexity: 1.2465
Epoch: 1	Batch: 100	Avg-Loss: 0.2231
Epoch: 1	Batch: 100	Training perplexity: 1.2500
Epoch: 1	Batch: 150	Avg-Loss: 0.2425
Epoch: 1	Batch: 150	Training perplexity: 1.2745
Epoch: 1	Batch: 200	Avg-Loss: 0.2225
Epoch: 1	Batch: 200	Training perplexity: 1.2492
Epoch: 1	Batch: 250	Avg-Loss: 0.2064
Epoch: 1	Batch: 250	Training perplexity: 1.2293
Epoch: 1	Batch: 300	Avg-Loss: 0.2210
Epoch: 1	Batch: 300	Training perplexity: 1.2473
Epoch: 1	Batch: 350	Avg-Loss: 0.2060
Epoch: 1	Batch: 350	Training perplexity: 1.2287
Epoch: 1	Batch: 400	Avg-Loss: 0.2229
Epoch: 1	Batch: 400	Training perplexity: 1.2497
Epoch: 1	Batch: 450	Avg-Loss: 0.1829
Epoch: 1	Batch: 450	Training perplexity: 1.2007
Epoch: 1	Batch: 500	Avg-Loss: 0.2003
Epoch: 1	Batch: 500	Training perplexity: 1.2217
Epoch: 1	Batch: 550	Avg-Loss: 0.1785
Epoch: 1	Batch: 550	Training perplexity: 1.1955
Epoch: 1	Batch: 600	Avg-Loss: 0.2068
Epoch: 1	Batch: 600	Training p

In [18]:
torch.save(model, 'hw3p2_best_model.pth')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
#epoch 13
numEpochs = 1
learningRate = 3e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
train_epoch_packed(model, train_loader, eval_loader)

Epoch: 1	Batch: 50	Avg-Loss: 0.2150
Epoch: 1	Batch: 50	Training perplexity: 1.2399
Epoch: 1	Batch: 100	Avg-Loss: 0.2229
Epoch: 1	Batch: 100	Training perplexity: 1.2496
Epoch: 1	Batch: 150	Avg-Loss: 0.2233
Epoch: 1	Batch: 150	Training perplexity: 1.2502
Epoch: 1	Batch: 200	Avg-Loss: 0.2101
Epoch: 1	Batch: 200	Training perplexity: 1.2338
Epoch: 1	Batch: 250	Avg-Loss: 0.1997
Epoch: 1	Batch: 250	Training perplexity: 1.2210
Epoch: 1	Batch: 300	Avg-Loss: 0.2148
Epoch: 1	Batch: 300	Training perplexity: 1.2396
Epoch: 1	Batch: 350	Avg-Loss: 0.1999
Epoch: 1	Batch: 350	Training perplexity: 1.2212
Epoch: 1	Batch: 400	Avg-Loss: 0.2005
Epoch: 1	Batch: 400	Training perplexity: 1.2220
Epoch: 1	Batch: 450	Avg-Loss: 0.2156
Epoch: 1	Batch: 450	Training perplexity: 1.2406
Epoch: 1	Batch: 500	Avg-Loss: 0.2052
Epoch: 1	Batch: 500	Training perplexity: 1.2278
Epoch: 1	Batch: 550	Avg-Loss: 0.2174
Epoch: 1	Batch: 550	Training perplexity: 1.2428
Epoch: 1	Batch: 600	Avg-Loss: 0.1986
Epoch: 1	Batch: 600	Training p

KeyboardInterrupt: 

In [13]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip3 install wget
%cd ctcdecode
!pip3 install .
%cd ..

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/home/ubuntu/ctcdecode
Processing /home/ubuntu/ctcdecode
  Created wheel for ctcdecode: filename=ctcdecode-0.4-cp36-cp36m-linux_x86_64.whl size=11241017 sha256=3d4d3d8c7da2975b48b0c7426163657bfaf03ab516a4e97bc96a678fb7180049
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8lkka2j/wheels/bc/c3/2c/f1c79bf1243976282df707f69771e696116f7bddf6a40edeb4
Successfully built ctcdecode
  Found existing installation: ctcdecode 0.4
    Uninstalling ctcdecode-0.4:
      Successfully uninstalled ctcdecode-0.4
/home/ubuntu


In [13]:
import torch
from ctcdecode import CTCBeamDecoder
decoder = CTCBeamDecoder(['$'] * num_classes, beam_width=100, log_probs_input=True)

In [14]:
class Testset(Dataset):
    def __init__(self, frames):
        self.frames=[torch.tensor(f) for f in frames]
    def __getitem__(self,i):
        time_steps = self.frames[i]
        return time_steps.to(device)
    def __len__(self):
        return len(self.frames)

# for packed_seqs, you want to return your data sorted by length
def collate_lines(seq_list):
    inputs = seq_list
    X_lens = torch.LongTensor([len(seq) for seq in inputs])
    X = pad_sequence(inputs)

    return X, X_lens

In [15]:
test_dataset = Testset(testX)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False, batch_size=32, collate_fn = collate_lines)

In [20]:
prediction = []

def predict(model, test_loader):
  
    with torch.no_grad():
        model.eval()
        model.to(device)

        for batch_num, (inputs, X_lens) in enumerate(test_loader):

            outputs, out_lens = model(inputs, X_lens)
            test_Y, _, _, test_Y_lens = decoder.decode(outputs.transpose(0, 1), out_lens)
            
            for i in range(len(test_Y)):
                # For the i-th sample in the batch, get the best output
                best_seq = test_Y[i, 0, :test_Y_lens[i, 0]]
                sequence = ''
                for c in best_seq:
                    if c != 0:
                        sequence += PHONEME_MAP[c-1] 
                prediction.append(sequence)
                
predict(model, test_loader)

In [21]:
import pandas as pd
result = pd.DataFrame()
result['Id'] = list(range(523))
result['Predicted'] = prediction
result.to_csv('zichenli_hw3p2_dropout_6.csv', encoding='utf-8', index=False)